In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [2]:
from weather_api import *

In [3]:
df = get_data("zakopane")

In [11]:
df.head()

,Kod stacji,Nazwa stacji,Rok,Miesiąc,Dzień,Średnie dobowe zachmurzenie ogólne [oktanty],Średnia dobowa prędkość wiatru [m/s],Średnia dobowa temperatura [°C],Średnia dobowe ciśnienie pary wodnej [hPa],Średnia dobowa wilgotność względna [%],Średnia dobowe ciśnienie na poziomie stacji [hPa],Średnie dobowe ciśnienie na pozimie morza [hPa],Status pomiaru PPPM,Suma opadu dzień [mm],Suma opadu noc [mm]
0,349190625,ZAKOPANE,2020,1,1,4.3,2.0,-1.7,4.0,73.1,929.5,0.0,8,0.0,0.0
1,349190625,ZAKOPANE,2020,1,2,4.4,1.3,-1.5,3.1,56.8,928.8,0.0,8,0.0,0.0
2,349190625,ZAKOPANE,2020,1,3,2.5,1.4,-1.1,3.8,68.8,922.4,0.0,8,0.0,0.0
3,349190625,ZAKOPANE,2020,1,4,7.6,3.6,0.8,5.4,83.6,917.5,0.0,8,1.9,6.5
4,349190625,ZAKOPANE,2020,1,5,7.9,2.8,-4.1,3.8,83.6,924.4,0.0,8,3.4,0.0


In [16]:
def normalize_particular_columns(df, column_names):
    for col_name in column_names:
        df.iloc[:,col_name]=(df.iloc[:,col_name]-df.iloc[:,col_name].mean())/df.iloc[:,col_name].std()

In [13]:
value = []
data = df.iloc[:,[14]]
data = data.to_numpy()
for row in data:
    if row == 0:
        value.append(0)
    elif row > 0:
        value.append(1)
df['deszcz w dzień']= value

value1 = []
data1 = df.iloc[:,[15]]
data1 = data1.to_numpy()
for row1 in data1:
    if row1 == 0:
        value1.append(0)
    elif row1 > 0:
        value1.append(1)
df['deszcz w nocy']= value

In [14]:
df

,Kod stacji,Nazwa stacji,Rok,Miesiąc,Dzień,Średnie dobowe zachmurzenie ogólne [oktanty],Średnia dobowa prędkość wiatru [m/s],Średnia dobowa temperatura [°C],Średnia dobowe ciśnienie pary wodnej [hPa],Średnia dobowa wilgotność względna [%],Średnia dobowe ciśnienie na poziomie stacji [hPa],Średnie dobowe ciśnienie na pozimie morza [hPa],Status pomiaru PPPM,Suma opadu dzień [mm],Suma opadu noc [mm],deszcz w dzień,deszcz w nocy
0,349190625,ZAKOPANE,2020,1,1,4.3,2.0,-1.7,4.0,73.1,929.5,0.0,8,0.0,0.0,0,0
1,349190625,ZAKOPANE,2020,1,2,4.4,1.3,-1.5,3.1,56.8,928.8,0.0,8,0.0,0.0,0,0
2,349190625,ZAKOPANE,2020,1,3,2.5,1.4,-1.1,3.8,68.8,922.4,0.0,8,0.0,0.0,0,0
3,349190625,ZAKOPANE,2020,1,4,7.6,3.6,0.8,5.4,83.6,917.5,0.0,8,1.9,6.5,1,1
4,349190625,ZAKOPANE,2020,1,5,7.9,2.8,-4.1,3.8,83.6,924.4,0.0,8,3.4,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,349190625,ZAKOPANE,2019,12,27,7.9,2.1,-2.0,4.8,91.3,920.5,0.0,8,4.6,6.5,1,1
543,349190625,ZAKOPANE,2019,12,28,8.0,2.6,-4.6,4.0,90.6,928.0,0.0,8,3.7,0.4,1,1
544,349190625,ZAKOPANE,2019,12,29,7.1,0.9,-7.3,3.2,88.3,933.2,0.0,8,1.1,0.0,0,0
545,349190625,ZAKOPANE,2019,12,30,6.0,1.3,-2.3,4.8,87.5,931.0,0.0,8,0.0,0.0,0,0


In [18]:
columns_to_normalize = [[6],[7],[9],[10]]
normalize_particular_columns(df, columns_to_normalize)

In [ ]:
df = result.drop(result.columns[11],axis = 1)
result.head()

In [ ]:
X = result[[result.columns[5],result.columns[6],result.columns[7],result.columns[8],result.columns[9],result.columns[10]]]
y = result[[result.columns[14]]]

In [ ]:
Xtrain, Xtest, ytrain, ytest  = train_test_split(X,y,test_size=0.2,random_state=75)

Model SVN 

In [ ]:
clf= svm.SVC(kernel='linear', C=1.0)
clf.fit(Xtrain,ytrain)

In [ ]:
ypred = clf.predict(Xtest)
clf_score = clf.score(Xtest,ytest)

In [ ]:
clf_score

In [ ]:
confusion_matrix(ytest,ypred)

Model logistic regresion

In [ ]:
logistic_regression= LogisticRegression()
logistic_regression.fit(Xtrain,ytrain)
y_pred_r=logistic_regression.predict(Xtest)
    
accur = accuracy_score(ytest, y_pred_r)

In [ ]:
accur

In [ ]:
confusion_matrix(ytest,y_pred_r)

Model KNN

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
knn.fit(Xtrain, ytrain)
y_pred_knn = knn.predict(Xtest)
accurKNN = accuracy_score(ytest, y_pred_knn)

In [ ]:
accurKNN

In [ ]:
confusion_matrix(ytest,y_pred_knn)